Usually, people bring up the total streaming amount to show how much did public like the music. 

but

I believe,  the total streaming amount is just showing off how many people it can reach.
There can be many songs that you might like, but you don't listen to because you don't know its presence

and like such, the streaming amount is not covering all the people that will like the music. 

There are songs that obtained its accessibility to population and reached high position and streaming amount, but collapsed right after its surge. 

Also there can be good songs that didn't reach out to people yet, but  possessing a potential to be popular.


Even the Gangnam style which had the potential to be globally popular,  stayed very low streaming amount before accidently got the access to global consumers.  

It obtained the access to global population at the end of 2012 and made a massive explosion. even now people come up with the horse riding dance

Its potential could not had been seen clearly through its total streaming before becoming popular.

Bringing up the total streaming value as a measurement may not be a correct way to see 

-----------------


In this kernel, Instead of the total streaming amount,  In order to witness and measure the true value of songs,  

The kernel is applying an idea that repeated consumption is a decent measurement to witness the consumer satisfaction.

We are going to sort music by how long it maintained its consumers, measuring the value only by people who have at least listened to it once, whether they move out or stay in, not the people who potentially like it, usually deemed as the total streaming amount. 

This way, we might have been able to anticipate the Gangnam style's potential even before it got its accessibility.  



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 

In [ ]:
spo = pd.read_csv('../input/spotifys-worldwide-daily-song-ranking/data.csv')
releasedate = pd.read_csv('../input/spotifys-daily-song-ranking-music-released-date/spotify_music_released_date.csv', header = None)

In [ ]:
releasedate = releasedate.dropna()
def fam(x):
    return datetime.strptime(x[:10] , "%Y-%m-%d") 
mj = releasedate[1].map(fam)
releasedate.iloc[:,1] = mj

modify the list of songs' released date into a datetime format.

In [ ]:
releasedate.head(20)

Since, due to the inherent limitation in the dataset, that the full charts are only between 2017 and 2018, To watch the entire streaming life cycle of a music, we are going to limit the sample for the kernel to only songs that are released between 2017-1-1 and 2017-4-1

*through this sampling process, 'New Rules' and 'Havana' are excluded. They were the true outliers and I really liked them which were released in the middle of 2017.  

In [ ]:
releasedate = releasedate[releasedate[1] >= datetime(2017,1,1)]
releasedate = releasedate[releasedate[1] <= datetime(2017,4,1)] 
tglist = list(releasedate[0])                                       
fg = lambda x : True if x in tglist else False
newspo = spo['Track Name'].map(fg) 

Here are the samples we are going to line them up

In [ ]:
spo[newspo].head(20)

Not using the raw streaming data, we are going deploy a trend function and re-create the data.
This is because the raw streaming data fluctuate too much

In [ ]:
finaldf = pd.DataFrame(index = tglist, columns = ['value','streaminglevel', 'startdate', 'entlen' ,'morethan70len', 'maxlen'])
for iname in tglist:
    testRickey = spo[spo['Track Name'] ==iname].groupby('Date').sum()
    testRickey = testRickey.reset_index()
    susul2 = testRickey.loc[:,['Date','Streams']]
    susul2['Ra'] = range(len(susul2))
    ym = np.array(susul2['Streams'])
    xm = np.array(susul2['Ra'])
    zm = np.polyfit(xm, ym, 30)                                                   
    fm = np.poly1d(zm)                     
    x_Me = np.linspace(xm[0], xm[-1], len(testRickey))
    y_Me = fm(x_Me)                         
    newdict = { 'x' : x_Me, 'y' : y_Me}                 
    newdictdf = pd.DataFrame(newdict)  
    paramss = 0.7
    L = newdictdf['y'] >= (newdictdf['y'].max()*paramss)     
    newdictdf['TF'] = L                                         
    finaldf.loc[iname,'value'] = L.sum()/len(testRickey)
    finaldf.loc[iname,'streaminglevel']= y_Me.max()/100000
    finaldf.loc[iname,'startdate'] = susul2.iloc[0,0]
    finaldf.loc[iname,'entlen'] =  len(testRickey)
    finaldf.loc[iname,'morethan70len'] =  L.sum()
    finaldf.loc[iname,'maxlen'] =  371   

'maxlen' is 371, the full length of the dataset from 2017 to 2018. 

'entlen' is the length the music stayed on the chart

'morethan70len' is the length  that streaming amount is more than 70 percent of each song's maximum.

'streaming level' is the maximum streaming amount. divided it by 100,000 to see it easily. 

'startdate' is the first date that the music showed up on the chart.
 
'value' is a morethan70len divided by the entlen


In [ ]:
finaldf.head(10)

Just to make it more clear, we are going to sample it again for those only featuring decent amount of streaming.   at least 200,000

This is to eliminate possible problems, that music with too low streaming amount maintain their max streaming  as a default everyday and It is unclear whether it is losing its consumers or not  

Music with more than 200,000 streamings everyday will be observed clearly. 

In [ ]:
finaldfsort = finaldf[(finaldf['streaminglevel'] > 2)]

Here is what we got!

In [ ]:
New_method_list = finaldfsort.sort_values(by = ['morethan70len','value'], ascending = False)
New_method_list.head(10)

Those songs are amazing!
Top 10 music maintained their consumers for almost the entire year. 

What if we sort music by its total streaming amount

In [ ]:
ordinary_method_list = spo[newspo].groupby('Track Name').sum().sort_values(by = 'Streams', ascending = False)
ordinary_method_list.head(10)

Let's compare both rankings with visualization

In [ ]:
fig, ax = plt.subplots(10, 2, figsize=(13,70))

In [ ]:
for num, iname in enumerate(list(New_method_list.head(10).index)):
    testRickey = spo[spo['Track Name'] ==iname].groupby('Date').sum()
    testRickey = testRickey.reset_index()
    susul2 = testRickey.loc[:,['Date','Streams']]
    susul2['Ra'] = range(len(susul2))
    ym = np.array(susul2['Streams'])
    xm = np.array(susul2['Ra'])
    zm = np.polyfit(xm, ym, 30)  
    fm = np.poly1d(zm)    
    x_Me = np.linspace(xm[0], xm[-1], len(testRickey))
    y_Me = fm(x_Me)
    newdict = { 'x' : x_Me, 'y' : y_Me}                 
    newdictdf = pd.DataFrame(newdict) 
    paramss = 0.7
    L = newdictdf['y'] >= (newdictdf['y'].max()*paramss)
    newdictdf['TF'] = L                                       
    ax[num,1].plot(xm,ym,'o', x_Me, y_Me)
    ax[num,1].set_title("lined up by how long it maintains {0} , Title = {1}".format(num,iname))

In [ ]:
for num, iname in enumerate(list(ordinary_method_list.head(10).reset_index()['Track Name'])):
    testRickey = spo[spo['Track Name'] ==iname].groupby('Date').sum()
    testRickey = testRickey.reset_index()
    susul2 = testRickey.loc[:,['Date','Streams']]
    susul2['Ra'] = range(len(susul2))
    ym = np.array(susul2['Streams'])
    xm = np.array(susul2['Ra'])
    zm = np.polyfit(xm, ym, 30)                                                      
    fm = np.poly1d(zm)                      
    x_Me = np.linspace(xm[0], xm[-1], len(testRickey))
    y_Me = fm(x_Me)                         
    newdict = { 'x' : x_Me, 'y' : y_Me}                 
    newdictdf = pd.DataFrame(newdict)  
    paramss = 0.7
    L = newdictdf['y'] >= (newdictdf['y'].max()*paramss)    
    newdictdf['TF'] = L                                        
    ax[num,0].plot(xm,ym,'o', x_Me, y_Me)
    ax[num,0].set_title("lined up by total streaming amount {0} , Title = {1}".format(num,iname))

In [ ]:
fig

What we see here is that most of the songs ranked high with total streaming value, didn't really maintained its consumers, they left out earlier than songs on the right side. 

The song named 'Congratulations' reached very high position by the total streaming amount and also ranked high by maintaing its consumers which is the best music I present. 

In this kernel, we assume that music only reach their established consumers immediately after release, and there is low possibility that new listeners comming in as time goes by.
In addition, we had filtered music which streaming level is more than 200,000, once again eliminating such cases.
So the longer it maintained its streaming amount the more those fans repeatedly listened to it, indicating that music is more valuable.
Those music could have made a global explosion if they had obtained an accessibility to more population.  



Some might argue that those songs ranked high with total streaming amount is still featuring absolutely more number of streaming amount compare to the songs filtered by the new method, so they worths much more than those songs.
But still, the fact that they lost their consumers who once approached, that is the point of the kernel. 

I admit that streaming amount is a combination of various  factors and this kernel is written on very strict limitations. The frame for the observation was limited to only one year from 2017 to 2018 
which led us to only sample songs only released between 2017-1-1 and 2017-4-1 excluding so many awesome songs. 

It could have been more awesome if we were able to observe all songs. 

But the idea, to rank songs by how long they maintained their consumers was worth a try. 

Thanks for reading. 
